# Ejercicio 3

3. Realizar un análisis de stopwords del contenido de texto de la Wikipedia. En este punto esperamos que analicen, dada la frecuencia de los términos que hay en la wikipedia cuales deberian ser considerados stop words. (⭐)

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_textos = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/contents_text_sample.csv", header=True, inferSchema=True)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
rdd_texto = df_textos.select("text").rdd

In [ ]:
rdd_texto = rdd_texto.filter(lambda x: x.text != None)

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk

nltk.download("punkt")
nltk.download("stopwords")
stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from operator import add
rdd_texto = rdd_texto.flatMap(lambda valor: [(x, 1) for x in word_tokenize(valor.text)]).reduceByKey(add)
rdd_texto.take(5)


[('s', 5730), ('1818.', 3), ('Trop', 51), ('>', 138212), ('Tropicos.org', 48)]

Si bien la gran mayoria de terminos que aparecen con freciencia (se eligio 4000 apariciones) pueden ser consideradas stopwords, hay palabras como "nombre, septiembre, abril, persona, edicion, etc" que debido a su amplio uso tienen alta frecuencia pero no por eso son stopwrds.

In [ ]:
stop_words = rdd_texto.filter(lambda x: x[1] > 4000)
stop_words.collect()

Si utilizamos de filtro una frecuencia aun mayor, podemos ver que la totalidad de terminos podrian ser considerados stopwords en este contexto.

In [ ]:
stop_words_mas_frecuencia = rdd_texto.filter(lambda x: x[1] > 60000)
stop_words_mas_frecuencia.collect()

[('>', 138212),
 ('}', 174922),
 ('del', 98812),
 ('los', 84631),
 (':', 133609),
 (')', 122476),
 ('por', 70049),
 ('un', 64638),
 ('la', 245462),
 ('REDIRECCIÓN', 69409),
 ('con', 61152),
 ("''", 246411),
 ('``', 85094),
 ('#', 102231),
 ('de', 742797),
 (']', 982543),
 ('(', 108643),
 ('[', 968719),
 ('.', 309446),
 ('en', 208003),
 ('y', 191136),
 ('el', 172811),
 ('que', 117909),
 ('{', 278744),
 ('<', 137597),
 ('ref', 64421),
 ('a', 114308),
 ('se', 68228)]

Por otro lado tambien podemos ver que hay palabras consideradas stopwords del español por una libreria, que no necesariamente tienen un alto numero de apariciones, algunas apareciendo solamente 1 o dos veces

In [ ]:
libreria = rdd_texto.filter(lambda x: x[0] in stopwords)
libreria.collect()

[('del', 98812),
 ('son', 8093),
 ('su', 46646),
 ('los', 84631),
 ('para', 38318),
 ('esto', 1757),
 ('hay', 2420),
 ('ante', 1857),
 ('antes', 2932),
 ('no', 22717),
 ('ya', 5958),
 ('algunas', 1591),
 ('estado', 3129),
 ('durante', 6489),
 ('tenía', 2097),
 ('estén', 122),
 ('estar', 1036),
 ('hubiera', 220),
 ('vuestro', 11),
 ('estando', 176),
 ('serían', 234),
 ('suyos', 57),
 ('esté', 208),
 ('mí', 180),
 ('tengamos', 19),
 ('os', 85),
 ('fuimos', 13),
 ('habíamos', 20),
 ('hayas', 21),
 ('estáis', 7),
 ('estés', 10),
 ('tendré', 5),
 ('tendréis', 1),
 ('vuestra', 27),
 ('seré', 13),
 ('tenida', 6),
 ('habidas', 3),
 ('estuvimos', 2),
 ('estabais', 2),
 ('hubiéramos', 2),
 ('seremos', 2),
 ('hubieseis', 1),
 ('estuvisteis', 1),
 ('estas', 1476),
 ('una', 54263),
 ('ha', 7269),
 ('por', 70049),
 ('un', 64638),
 ('la', 245462),
 ('esta', 6793),
 ('será', 768),
 ('desde', 7758),
 ('otro', 2450),
 ('habrán', 8),
 ('han', 3301),
 ('otros', 6104),
 ('muy', 4126),
 ('has', 244),
 ('alg

Las palabras que podriamos considerar si o si stopwords para wikipedia, sumado a aquellas con mas de 60000+ apariciones, podrian ser las q cumplen ambas condiciones (tener mas de 4000 apariciones pero tambien aparecer en la libreria utilizada)

In [ ]:
cruce = libreria.join(stop_words)

resultado = cruce.fullOuterJoin(stop_words_mas_frecuencia).map(lambda x: x[0])
resultado.collect()

['del',
 'son',
 'su',
 'los',
 'para',
 'no',
 'ya',
 'durante',
 '>',
 '}',
 'una',
 'ha',
 'por',
 'un',
 'la',
 'esta',
 'desde',
 'otros',
 'muy',
 'sin',
 'contra',
 'fueron',
 ':',
 ')',
 'REDIRECCIÓN',
 'pero',
 'con',
 'como',
 'también',
 'lo',
 'sus',
 'le',
 'cual',
 'sobre',
 "''",
 '``',
 '#',
 'de',
 'o',
 'las',
 'era',
 'entre',
 'más',
 'es',
 'cuando',
 'está',
 ']',
 '(',
 'en',
 'y',
 'el',
 'que',
 'hasta',
 'al',
 'este',
 '[',
 '.',
 'a',
 'donde',
 'se',
 'e',
 'fue',
 'tiene',
 'había',
 'uno',
 '{',
 '<',
 'ref']

# Ejercicio 28

28. El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones (⭐⭐)


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_contents = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/contents.csv", header=True, inferSchema=True)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Se que mi archivo ya tiene solamente las versiones finales (por enunciado)


---


USUARIOS:

Se que no tener titulo no es necesariamente descalificante para la fila, porque puede ser una edicion en progreso, lo unico que no puede pasar es todos los valores nan o que no hay revisor_id, puesto que nada me asegura que todos los nan pertenezcan a un mismo editor, por lo que se los descarta.  


---
Me interesa cuantas veces aparece cada usuario, puesto que cada linea es una edicion de contenido, y el comentario que se realizo durante esa edicion.

In [ ]:
rdd_contents = df_contents.select("revisor_id","revisor_comment").rdd

In [ ]:
#Filtro ids de revisor nans
rdd_contents = rdd_contents.filter(lambda x: x.revisor_id != None)

In [ ]:
def categorizar_usuario (cant_comentarios):
  if (cant_comentarios == 1):
      return "1"
  elif (cant_comentarios > 1 and cant_comentarios < 100):
      return ">10"
  else:
    return ">100"

In [ ]:
comento = True
no_comento = False

def categorizar_comentario (comentario):
  if (comentario == None):
    return no_comento
  return comento



*  Cuento cuantas ediciones hace cada usuario





In [ ]:
comentarios_por_usuario = rdd_contents.map(lambda x: (x.revisor_id,1)).countByKey()


*  En mi rdd, solo me quedo con la categoria del usuario (1, >10 comentarios, > 100 comentarios) y del comentario (no comento, comento). De esta forma me queda un listado que indica si en mi edicion se agrego o no un comentario y a su vez ya se a cual de mis tres grupos pertenece.

*   Luego agrupo por key de forma tal que me van a quedar solo 6 elementos




In [ ]:
#((cat_usuario, cat_comentario), 1)
rdd_contents = rdd_contents.map(lambda x: ((categorizar_usuario(comentarios_por_usuario[x.revisor_id]), categorizar_comentario(x.revisor_comment)), 1)  )
rdd_contents = rdd_contents.countByKey()
rdd_contents

defaultdict(int,
            {('>100', True): 3377355,
             ('>10', False): 42163,
             ('>10', True): 150895,
             ('>100', False): 343688,
             ('1', True): 18427,
             ('1', False): 6349})

In [ ]:
total_1 = rdd_contents[('1', comento)] + rdd_contents[('1', no_comento)]
sin_comentar_1 = rdd_contents[('1', no_comento)]
porcentaje_1 =  sin_comentar_1/total_1 * 100

total_10 = rdd_contents[('>10', comento)] + rdd_contents[('>10', no_comento)]
sin_comentar_10 = rdd_contents[('>10', no_comento)]
porcentaje_10 =  sin_comentar_10/total_10 * 100

total_100 = rdd_contents[('>100', comento)] + rdd_contents[('>100', no_comento)]
sin_comentar_100 = rdd_contents[('>100', no_comento)]
porcentaje_100 =  sin_comentar_100/total_100 * 100

resultado = (('1', porcentaje_1), ('>10', porcentaje_10),('>100', porcentaje_100))
resultado = sc.parallelize(resultado,1)
resultado.collect()

[('1', 25.62560542460446),
 ('>10', 21.839550808565303),
 ('>100', 9.236335081319941)]

# Ejercicio 35

35. ¿Cuál es el mínimo que ha durado desde su registro un usuario bloqueado en la plataforma? (⭐⭐)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_logs = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/logs.csv", header=True, inferSchema=True)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Se que el logtype newuserts tiene la accion create, pero que otros logtypes tambien tienen create, por lo que necesito filtrar por ambos.

Por otro lado el logtype block tiene reblock, block y unblock, pero ningun otro logtype tiene block y reblock, por lo que solo me interesa action.

In [ ]:
rdd_creacion = df_logs.select("logtype","action","timestamp","title").rdd
rdd_bloqueos = df_logs.select("action","timestamp","title").rdd

* Filtro los registros donde el usuario creado es nan, y me quedo solo con los newusers creados
* Filtro para quedarme solo los action de block y reblock y que sepa q usuario se blockeo (filtro nans)

In [ ]:
rdd_creacion = rdd_creacion.filter(lambda x: x.title != None and x.logtype == 'newusers' and x.action == 'create')
rdd_bloqueos = rdd_bloqueos.filter(lambda x: x.title != None and (x.action == 'block' or x.action == 'reblock'))

BLOQUEOS

Agrupo las veces que bloquearon a cada usuario

In [ ]:
rdd_bloqueos = rdd_bloqueos.map(lambda x: (x.title, x.timestamp))
rdd_bloqueos = rdd_bloqueos.groupByKey(1)
rdd_bloqueos = rdd_bloqueos.mapValues(list)

CREACIONES

Me lo guardo como clave,valor, donde la clave es el usuario y el valor cuando se creó

In [ ]:
rdd_creacion = rdd_creacion.map(lambda x: (x.title,x.timestamp))

Creo un RDD para trabajar que tenga todos los datos.

In [ ]:
from datetime import datetime
from datetime import timedelta
cero = timedelta(seconds = 0)

def minimo_tiempo(creacion, lista_bloqueos):

  creacion = creacion
  tiempo_creacion = datetime.strptime(creacion,"%Y-%m-%dT%H:%M:%SZ")

  primer_bloqueo = lista_bloqueos[0]
  tiempo_primer_bloqueo =  datetime.strptime(primer_bloqueo,"%Y-%m-%dT%H:%M:%SZ")

  min = tiempo_primer_bloqueo - tiempo_creacion


  for bloqueo in lista_bloqueos:
    tiempo_bloqueo_actual = datetime.strptime(bloqueo,"%Y-%m-%dT%H:%M:%SZ")
    diferencia = tiempo_bloqueo_actual - tiempo_creacion
    if ((diferencia <= min and diferencia > cero) or min < cero):
      min = diferencia

  return min

Los usuarios bloqueados son 299.727 mientras que los creados son 4.742.390, por lo que los filtro en base a los bloqueados.

A su vez debido a la cantidad de datos, elijo realizar un diccionario y acceder al mismo en vez de utilizar un join con ambos rdd lo cual disminuye considerablemente el tiempo (debido a que bloqueados no cuenta con gran tamaño frente a creados).

In [ ]:
dic = rdd_bloqueos.collectAsMap()

Me quedo solo con los creados que fueron bloqueados alguna vez, calculo su tiempo minimo entre creacion y bloqueo (evitando el negativo de ser posible), borro los negativos que quedan de forma inevitable y me quedo con el menor valor.

In [ ]:
resultado = rdd_creacion.filter(lambda x: x[0] in dic.keys())
resultado = resultado.map(lambda x: (x[0], minimo_tiempo(x[1], dic[x[0]])))
resultado = resultado.filter(lambda x: x[1] > cero)
resultado.reduce(lambda x,y: x if x[1]<y[1] else y)


('Usuario:Unfollow Ariiana Grande 03.07.2021 7', datetime.timedelta(seconds=4))

# Ejercicio 38

Si decimos que la ubicación de un usuario es el promedio de la latitud y longitud de los contenidos geolocalizados para los cuales editó la última versión (ignorar usuarios que no editaron contenido geolocalizado). ¿Cuáles son los dos usuarios más cercanos? (⭐⭐⭐)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_contents = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/contents.csv", header=True, inferSchema=True)
df_geo_tags = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/geo_tags.csv", header=True, inferSchema=True)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Se que el logtype newuserts tiene la accion create, pero que otros logtypes tambien tienen create, por lo que necesito filtrar por ambos.

Por otro lado el logtype block tiene reblock, block y unblock, pero ningun otro logtype tiene block y reblock, por lo que solo me interesa action.

In [ ]:
rdd_usuarios = df_contents.select("id","revisor_id").rdd
rdd_geo = df_geo_tags.select("gt_page_id","gt_lat","gt_lon").rdd

Filtro aquellas filas que no tengan la informacion suficiente para ser consideradas en la busqueda.

In [ ]:
rdd_usuarios = rdd_usuarios.filter(lambda x: x.revisor_id != None and x.id != None)
rdd_geo = rdd_geo.filter(lambda x: x.gt_page_id != None and x.gt_lat != None and x.gt_lon != None)

In [ ]:
rdd_usuarios = rdd_usuarios.map(lambda x: (x.id,x.revisor_id))
rdd_geo = rdd_geo.map(lambda x: (str(x.gt_page_id),(x.gt_lat,x.gt_lon)))

Me quedo solo con los contenidos geolocalizados de los cuales tengo un usuario asociado y viceversa (inner)

In [ ]:
rdd_datos = rdd_usuarios.join(rdd_geo)

Me quedo solo con (usuario, (lat,long))

In [ ]:
rdd_datos = rdd_datos.map(lambda x: (x[1][0],x[1][1]))


Guardo todo como (usuario, lista de lat y longitudes)

In [ ]:
rdd_datos = rdd_datos.groupByKey()
rdd_datos = rdd_datos.mapValues(list)

In [ ]:
def promedio(tuplas):
  resultado_lat = 0
  resultado_lon = 0
  cantidad = 0
  for tupla in tuplas:
    resultado_lat += tupla[0]
    resultado_lon += tupla[1]
    cantidad += 1
  return (resultado_lat/cantidad, resultado_lon/cantidad)

Consigo la ubicacion de cada usuario sacando el promedio de sus lat y long de contenidos

In [ ]:
rdd_datos = rdd_datos.map(lambda x: (x[0], promedio(x[1])))

Agrupo aquellos usuarios que tengan la misma ubicacion y se que esos son los que estan mas cercanos entre si.

In [ ]:
rdd_datos = rdd_datos.map(lambda x: (x[1],x[0]))
rdd_datos = rdd_datos.groupByKey().mapValues(list)
rdd_datos = rdd_datos.filter(lambda x: len(x[1]) > 1)

Los usuarios mas cercanos son cualquiera pertenecientes a una misma ubicacion:
((ubicacion),[lista usuarios])

In [ ]:
rdd_datos.collect()

[((43.31833333, 11.33138889), ['4451795', '6217219']),
 ((-33.35, -70.51666667), ['6092354', '5894133']),
 ((3.521642, -76.417036), ['6093271', '6289014']),
 ((35.68972222, 139.69222222), ['6190329', '3899149']),
 ((42.14166667, -7.97611111), ['6269272', '6269271']),
 ((19.33202222, -99.19635833), ['5521795', '2045384']),
 ((13.70853611, -89.24098889), ['6011069', '6024005']),
 ((39.95694444, -3.49666667), ['450789', '1682668']),
 ((-34.60180556, -58.39944444), ['4999877', '6304273']),
 ((22.767, 108.388), ['4898811', '5642895']),
 ((10.45, -64.16666667), ['1981184', '6279043'])]